<center><img src="unicycle_diagram.png" width="300"></center>

The equation of motion of the lumped-parameter unicycle model constrained in a plane can be derived using Euler-Lagrange equations (which is beyond our scope). The unicycle is assumed to have a single motor connecting the wheel and the rod with inertia, which we take as a crude model of the person riding the unicycle. Neglecting all friction we find the equation of motions as

$$
\begin{align}
    \ddot\theta(I_b + \ell^2 m_b) + \ddot\phi \ell d m_b \cos\theta - \ell g m_b \sin\theta &= \tau \\
    \ddot\phi ( I_w + d^2(m_b + m_w) + \ddot\theta \ell d m_b \cos\theta - \dot\theta^2 \ell d m_b \sin\theta &= -\tau
\end{align}
$$
or in matrix form
$$
\begin{equation}
    \underbrace{\begin{bmatrix}I_b + \ell^2 m_b & \ell d m_b \cos\theta \\ \ell d m_b \cos\theta & I_w + d^2(m_b + m_w)\end{bmatrix}}_M \underbrace{\begin{bmatrix}\ddot\theta \\ \ddot\phi\end{bmatrix}}_q + \underbrace{\begin{bmatrix}-\ell g m_b \sin\theta \\ -\dot\theta^2\ell m_b \sin\theta \end{bmatrix}}_c = \underbrace{\begin{bmatrix}\tau\\ -\tau\end{bmatrix}}_f
\end{equation}
$$

It's important to realize there are two degrees of freedom of our system, while there is only one actuator (one motor, or set of pedalling legs), so this system is actually under-actuated and we will be forced to leave one degree of freedom uncontrolled. Since the primary objective is to not fall over, we will choose to control body pitch angle $\theta$ and leave $\phi$ uncontrolled. In other words, we won't be able to control the position of the unicycle with respect to the ground and the body pitch at the same time. However, as you will soon see for yourself, we can at least decide to go forwards or backwards by leaning forwards or backwards.

In order to decouple the equation of motion, we can pre-multiply the above equation with $M^{-1}$.
$$
\begin{equation}
    \begin{bmatrix}\ddot\theta \\ \ddot\phi\end{bmatrix} = \frac{1}{\mathrm{det}[M]} \begin{bmatrix} I_w + d^2(m_b + m_w) & -\ell d m_b \cos\theta \\ -\ell d m_b \cos\theta & I_b + \ell^2 m_b \end{bmatrix} \begin{bmatrix}\tau + \ell g m_b \sin\theta \\ -\tau + \dot\theta^2\ell m_b \sin\theta\end{bmatrix}
\end{equation}
$$
where
$$
\begin{equation}
    \mathrm{det}[M] = I_b I_w + I_b d^2 (m_b + m_w) + I_w \ell^2 m_b + \ell^2 d^2 m_b (m_b \sin^2\theta + m_w)
    \end{equation}
$$

Then we can find the following differential equation describing the dynamics of body pitch
$$
\begin{equation}
    \ddot\theta = \frac{1}{\mathrm{det}[M]}\left( I_w \tau  + d^2(m_b + m_w)\tau  + \ell d m_b \tau \cos\theta  - \ell^2 d m_b^2 \dot\theta^2 \cos\theta \sin\theta + I_w\ell g m_b \sin\theta +  \ell  d^2  m_b(m_b + m_w) g \sin\theta \right)
\end{equation}
$$

In order to be able to proceed further with the tools we have learned so far in ME304, we have to linearize this relation about an operating point. However, as we have just found out, the unicycle has highly non-linear dynamics (there are lots of sine and cosine terms), so the results we find may not be very useful. Let's assume $\theta \approx 0$ so $\sin\theta \approx 0$ and $\cos\theta \approx 1$, then we can write
$$
\begin{equation}
    \ddot\theta \approx A\tau
\end{equation}
$$
where
$$
\begin{equation}
    A = \frac{I_w  + d^2(m_b + m_w) + \ell d m_b}{I_b I_w + I_b d^2 (m_b + m_w) + I_w \ell^2 m_b + \ell^2 d^2 m_b m_w}
\end{equation}
$$

which is just a constant since we should know the system parameters from beforehand.

In Laplace domain, this is
$$
\begin{equation}
    \Theta(s) = \frac{A}{s^2} T(s)
\end{equation}
$$

We've just found the transfer function of body pitch when the body is more or less perpendicular. Let's think about this for a second. If we check the Hurwitz Criterion, we can see that this system is not stable straight away, since some of the coefficients of the characteristic polynomial are zero (such as the coefficients of the $s^1$ and $s^0$ terms). We haven't added a controller yet, however we can safely say $\theta$ will go out of control if an impulse torque is given to the system by the motor. Since the system has failed Hurwitz Criterion, we don't even need to check Routh Criterion (it is sure to fail, as exercise, try to show it). Physically, this makes sense, unless we do something about it, the unicycle will topple over since it's basically an inverted pendulum.

Now let's consider adding a simple controller to this system to make it stable. [WIP]

First of all we will import some libraries we will be using. Unicycle game library was written for this document and it will help us in simulating and animating the unicycle in real time.

In [12]:
import unicycle_game as ug
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Feel free to play with the system and control parameters, or specify new initial conditions. Even better, you can try to write your own controller! Also please note that an arbitrary viscous damping applied on the wheel has been added to the dynamics to make it a bit more difficult to accidentally reach light speed during simulation. It's probably unrealistic and you can remove it if you wish. Furthermore, in our simulation we won't be giving any consideration to physical limitations of our motor (for example, we are assuming it can produce any forcing instantaneously anytime we wish, which is impossible in real life).

In [21]:
# system parameters
g = 9.81 # [m s^-2] gravity
l = 1 # [m] distance of the load from the center of the wheel
d = 1 # [m] diameter of the wheel
mb = 70 # [kg] lumped mass of the load
mw = 5 # [kg] lumped mass of the wheel
Ib = 1/3 * mb * l**2 # [kg m^2] rotational inertia of the load
Iw = mw * (d/2)**2 # [kg m^2] rotational inertia of the wheel
b = 30 # [N s/m] damping coefficient for the wheel (assuming viscous damping)
# control parameters
kp = 1000 # proportional gain
kv = 2 * kp**0.5 # velocity gain
# initial conditions
y0 = np.array([0, 0]) # initial position
dy0 = np.array([0, 0]) # initial velocity

# compute d2y
def compute_dynamics(y, dy, f):
    M = np.array([[Ib + l**2 * mb, l * d * mb * np.cos(y[0])], [l * d * mb * np.cos(y[0]), Iw + d**2 * (mb + mw)]])
    # eliminate the last term below to disable viscous damping
    c = np.array([-l * g * mb * np.sin(y[0]), -dy[0]**2 * l * d * mb * np.sin(y[0])]) + np.array([0, b * dy[1]]) # * 0 
    d2y = np.linalg.inv(M).dot(f - c)
    return d2y

# controller for theta, note that there is only one actuator
def control_theta(y, dy, th_desired, dth_desired):
    # P-controller
    
    # PID-controller
    
    
    # Here's another controller which uses velocity feedback, aswell
    th_error = th_desired - y[0]
    dth_error = dth_desired - dy[0]
    tau = kp * th_error + kv * dth_error
    f = np.array([tau, -tau]) # motor is connected between the two generalized coordinates
    return f

# update unicycle 
def update_unicycle(y, dy, th_desired, dth_desired, dt):
    f = control_theta(y, dy, th_desired, dth_desired)
    d2y = compute_dynamics(y, dy, f)
    # semi-implicit euler integration
    dy = dy + d2y * dt
    y = y + dy * dt
    return y, dy, d2y

We can now call our game and simulate the unicycle using our model and controller. You can play with the target $\theta$ in real time and see how the controller performs. You can change the dynamics and the controller in real time using the above coding block aswell, try changing the control gains while the simulation is running and see for yourself. Also, if your computer/browser can't keep up with the default 30 FPS (frames per second), you can try reducing the target FPS (and since they are tied together in this simulation, the updates per second).

In [22]:
try: # try to stop the game first in order to avoid creating redundant threads
    game.stop()
except NameError:
    pass
game = ug.Game(y0, dy0, update_unicycle, d, l, th0_desired=1/180*np.pi, dth0_desired=0, fps=30)
game.start()
display(game.canvas)

MultiCanvas(height=300, width=600)

You can use the following to stop the simulation at any time (however you won't be able to resume it due to how Python threads work, so you will have to start the simulation from the beginning).

In [9]:
game.stop()

In order to observe state variable histories, we can quickly draw some plots referring to our game object at any time (but they won't update in real time).

In [23]:
fig = make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=game.t_history, y=game.ydyd2y_history[:,0], name='$\\theta$'), row=1, col=1)
fig.add_trace(go.Scatter(x=game.t_history, y=game.th_desired_history, name='$\\theta_d$'), row=1, col=1)
fig.add_trace(go.Scatter(x=game.t_history, y=game.ydyd2y_history[:,2], name='$\\dot\\theta$'), row=2, col=1)
fig.add_trace(go.Scatter(x=game.t_history, y=game.ydyd2y_history[:,4], name='$\\ddot\\theta$'), row=3, col=1)
fig.update_xaxes(title_text="time (s)", row=3, col=1)
fig.show()
fig = make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=game.t_history, y=game.ydyd2y_history[:,1] * d, name='$x$'), row=1, col=1)
fig.add_trace(go.Scatter(x=game.t_history, y=game.ydyd2y_history[:,3] * d, name='$\\dot{x}$'), row=2, col=1)
fig.add_trace(go.Scatter(x=game.t_history, y=game.ydyd2y_history[:,5] * d, name='$\\ddot{x}$'), row=3, col=1)
fig.update_xaxes(title_text="time (s)", row=3, col=1)
fig.show()